In [10]:
from neon.backends import gen_backend
from neon.data import DataIterator, Text, load_text
from neon.initializers import Uniform, GlorotUniform
from neon.layers import GeneralizedCost, LSTM, Affine, Dropout, LookupTable, RecurrentSum
from neon.models import Model
from neon.optimizers import Adagrad
from neon.transforms import Logistic, Tanh, Softmax, CrossEntropyMulti, MeanSquaredMetric, MeanSquared
from neon.callbacks.callbacks import Callbacks
from neon.util.argparser import NeonArgparser
import numpy as np
import os
import cPickle as pickle
data_root = os.path.expanduser("~") + '/data/CSE255/'

In [2]:
# dummy class for arguments
class Args():
    pass
args = Args()

# the command line arguments 
args.backend         = 'gpu'
args.batch_size      = 128
args.epochs          = 2

args.config          = None
args.data_dir        = '/home/linuxthink/nervana/data'
args.datatype        = np.float32
args.device_id       = 0
args.evaluation_freq = 1
args.history         = 1
args.log_thresh      = 40
args.logfile         = None
args.model_file      = None
args.no_progress_bar = False
args.output_file     = '/home/linuxthink/nervana/data/neonlog.hd5'
args.progress_bar    = True
args.rng_seed        = 0
args.rounding        = False
args.save_path       = None
args.serialize       = 0
args.verbose         = 0

In [3]:
num_epochs = args.epochs

# hyperparameters from the reference
batch_size = 128
clip_gradients = True
gradient_limit = 15
vocab_size = 20000
sentence_length = 100
embedding_dim = 128
hidden_size = 128
reset_cells = True

# setup backend
be = gen_backend(backend=args.backend,
                 batch_size=batch_size,
                 rng_seed=args.rng_seed,
                 device_id=args.device_id,
                 default_dtype=args.datatype)

In [4]:
# make dataset
path = load_text('imdb', path=args.data_dir)
(X_train, y_train), (X_test, y_test), nclass = Text.pad_data(
    path, vocab_size=vocab_size, sentence_length=sentence_length)

print "Vocab size - ", vocab_size
print "Sentence Length - ", sentence_length
print "# of train sentences", X_train.shape[0]
print "# of test sentence", X_test.shape[0]

train_set = DataIterator(X_train, y_train, nclass=1)
valid_set = DataIterator(X_test, y_test, nclass=1)

Vocab size -  20000
Sentence Length -  100
# of train sentences 20000
# of test sentence 5000


In [40]:
raw_data = pickle.load(open(path, "rb"))

In [50]:
type(raw_data)

tuple

In [41]:
print(len(raw_data[0]), len(raw_data[1]))

(25000, 25000)


In [48]:
len(raw_data[0][3]) 

330

In [13]:
imdb_word_to_idx = pickle.load(open(data_root + "imdb.dict.pkl", "rb"))
imdb_idx_to_word = dict()
for word, idx in imdb_word_to_idx.iteritems():
    imdb_idx_to_word[idx] = word

In [47]:
idx_sentence = raw_data[0][0]
word_sentence = [imdb_idx_to_word[word] for word in idx_sentence]
' '.join(word_sentence)

'we really enjoyed grey owl : a simple tale well told in classic attenborough fashion : a little over-romanticized , with archetypes , humor , and a stress of dignity and values . &lt; br / &gt; &lt; br / &gt; beautifully shot and told at a nice pace this is the true story of archie , an englishman who turned native indian , and went to live and trap in canada . &lt; br / &gt; &lt; br / &gt; solid performances from all makes this film with a message easy viewing . &lt; br / &gt; &lt; br / &gt; two of the stars of the film are without a doubt the cutest we have ever seen - and the message is a good one with its ecoleanings . it must have been great to meet or read grey owl in the 1930s , a unique character and this is a fitting film tribute .'